# deep-audiobook-tuner
A system that generates an apt, emotionally pertinent, unique musical score for an audiobook automatically based on the current narrative for the purpose of ameliorating user-experience while being accurate, cost-efficient, and time saving.

### Dependencies

In [1]:
import os
import sys
sys.path.append(os.path.abspath("../"))
import tensorflow as tf
from deepaudiobooktuner.deep_audiobook_tuner import *

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Create an instance of the deepAudiobookTuner Class

In [3]:
dat = deepAudiobookTuner()

### Initializing the instance withe audiobook path

In [4]:
dat.initialize(audiobook_path = r'C:\Users\Lder\Desktop\AudioBooks\Alice_in_Wonderland_chap_1 (mp3cut.net).mp3')


Creating temporary directory.
----Temporary directory created.

Loading assets.
----Loaded audio model assets. Time taken: 1.8 s
----Loaded text model assets. Time taken: 10.5 s
----Loaded music model assets. Time taken: 1.6 s
----Setup IBM transcription service. Time taken: 0.0 s

Converting mp3 to wav
----Converted to wav. Time taken: 0.2 s

Segmenting audiobook
----Segmented audio file. Time taken: 4.6 s


### Performing Sentiment analysis by invoking the analyzeSentiments() function

In [5]:
dat.analyzeSentiments()



Performing sentiment analysis

Processing clip 1:
----Text sentiment analysis
--------Transcription complete. Time taken: 20.6 s
--------Text analysis complete. Time taken: 5.0 s
----Audio sentiment analysis
--------Audio analysis complete. Time taken: 1.4 s
----Predicting final emotion
----Clip 1 processed. Time taken: 27.1 s

Processing clip 2:
----Text sentiment analysis
--------Transcription complete. Time taken: 17.0 s
--------Text analysis complete. Time taken: 0.3 s
----Audio sentiment analysis
--------Audio analysis complete. Time taken: 1.3 s
----Predicting final emotion
----Clip 2 processed. Time taken: 18.6 s

Processing clip 3:
----Text sentiment analysis
--------Transcription complete. Time taken: 18.6 s
--------Text analysis complete. Time taken: 0.3 s
----Audio sentiment analysis
--------Audio analysis complete. Time taken: 1.3 s
----Predicting final emotion
----Clip 3 processed. Time taken: 20.2 s

Processing clip 4:
----Text sentiment analysis
--------Transcription c

### Generating Music by invoking the generateMusic() function

In [6]:
dat.generateMusic(music_emotions=["Angry", "Happy", "Neutral", "Sad"])



Generating music


----generated Angry clip. Time taken: 109.1 s


----generated Happy clip. Time taken: 69.8 s


----generated Neutral clip. Time taken: 126.4 s


----generated Sad clip. Time taken: 87.5 s
----
Music Generation Complete. Time taken: 396.4 s


In [7]:
for emotion in dat.songs:
    print(emotion)
    dat.songs[emotion].stream.show('midi')

Angry


Happy


Neutral


Sad


In [8]:
regen = ["Angry"]

In [9]:
dat.generateMusic(music_emotions = regen)



Generating music


----generated Angry clip. Time taken: 118.8 s
----
Music Generation Complete. Time taken: 122.6 s


In [10]:
dat.music_dict

{'angry': 'C:\\Users\\Lder\\Documents\\MAJOR_PROJ\\Github\\deep-audiobook-tuner\\assets\\temp\\Alice_in_Wonderland_chap_1 (mp3cut.net)-1619008710.060844\\music_clips/Angry.mp3',
 'happy': 'C:\\Users\\Lder\\Documents\\MAJOR_PROJ\\Github\\deep-audiobook-tuner\\assets\\temp\\Alice_in_Wonderland_chap_1 (mp3cut.net)-1619008710.060844\\music_clips/Happy.mp3',
 'neutral': 'C:\\Users\\Lder\\Documents\\MAJOR_PROJ\\Github\\deep-audiobook-tuner\\assets\\temp\\Alice_in_Wonderland_chap_1 (mp3cut.net)-1619008710.060844\\music_clips/Neutral.mp3',
 'sad': 'C:\\Users\\Lder\\Documents\\MAJOR_PROJ\\Github\\deep-audiobook-tuner\\assets\\temp\\Alice_in_Wonderland_chap_1 (mp3cut.net)-1619008710.060844\\music_clips/Sad.mp3'}

In [ ]:
for emotion in dat.songs:
    print(emotion)
    dat.songs[emotion].stream.show('midi')

### Generating the final soundtrack and overlaying it on the audiobook

In [12]:
dat.generateSoundtrack()

### Final audiobook generated by DAT

In [13]:
dat.final_audiobook

### Deleting the temperory directory once the user is done

In [ ]:
dat.deleteTempDirectory()

In [ ]:
dat.emotions

In [ ]:
dat.text_emotions_list

In [ ]:
dat.transcriptions[9]

In [ ]:
dat.songs